In [1]:
import time
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
from sklearn.metrics import brier_score_loss

timestr = time.strftime("%Y%m%d-%H%M%S")
dataset = pd.read_csv('new_data/dataset_full.csv')

dataset = pd.get_dummies(dataset)
saved_cols = dataset.columns
ace_col=0
for col in dataset.columns:
    if(col == 'win_reason_ace'):
        break
    ace_col+=1
dataset = dataset.replace(np.nan, 0)
dataset = dataset.to_numpy()

data = []
targets = []

i=0
while(i<int(dataset.shape[0])):
    data.append([])
    i+=1
i=0
while(i<dataset.shape[0]):
    
    ball_round = list()
    j=1
    while(j<dataset.shape[1]-2):
        ball_round.append(dataset[i,j])
        j+=1
    targets.append(dataset[i,j]) #team a wins label column
    
    data[i] = ball_round
    
    i+=1

#print(np.asarray(data))
data=np.asarray(data)
targets=np.asarray(targets)
#targets = targets.reshape(-1,1)

print(data.shape)

(2429, 53)


In [3]:
split_point = 2087

train_x = data[0:split_point]
train_y = targets[0:split_point]

#n_classes = len(np.unique(train_y))

train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], 1))
train_y = train_y.reshape(-1,1)

#remove win or lose reason labels (col 38 is first win reason)
train_x = train_x[:,:ace_col+1]
#for i in range(train_x.shape[0]):
#    train_x[i,56:] = 0

#remove all rounds with aces
a = train_x.shape[0]
i=0
while(i<a):
    if(train_x[i,ace_col] == 0):
        i+=1
        continue
    else:
        train_x = np.delete(train_x, i, 0)
        train_y = np.delete(train_y, i, 0)
        a-=1
        if(i<=split_point):
            split_point-=1
        continue
train_x = train_x[:,:ace_col]

In [4]:
def encoder(inputs, head_size, num_heads, ff_dim, dropout=0): #normalize, attention, normalize, feedforward, normalize
    # Normalization and Attention
    #mask = np.ones((inputs.shape[1],1))
    #mask[56:] = 0
    #inputs = tf.boolean_mask(inputs, mask, axis=None, name='boolean_mask')
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = tf.keras.layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(x, x)
    x = tf.keras.layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(res)
    x = tf.keras.layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = tf.keras.layers.Dropout(dropout)(x)
    x = tf.keras.layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    
    normalized_out = x + res
    return normalized_out


In [5]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = tf.keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = encoder(x, head_size, num_heads, ff_dim, dropout)

    x = tf.keras.layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = tf.keras.layers.Dense(dim, activation="relu")(x)
        x = tf.keras.layers.Dropout(mlp_dropout)(x)
    outputs = tf.keras.layers.Dense(1, activation="sigmoid")(x)
    return tf.keras.Model(inputs, outputs)


In [6]:
input_shape = train_x.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(loss=tf.losses.MeanSquaredError(),
                optimizer=tf.optimizers.Adam(),
                metrics=[tf.metrics.MeanAbsoluteError(), tf.metrics.AUC(), 'binary_accuracy'])
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    train_x,
    train_y,
    validation_split=0.2,
    epochs=200,
    batch_size=32,
    callbacks=callbacks,
)
#model.evaluate(x_test, y_test, verbose=1)


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 38, 1)]      0           []                               
                                                                                                  
 layer_normalization (LayerNorm  (None, 38, 1)       2           ['input_1[0][0]']                
 alization)                                                                                       
                                                                                                  
 multi_head_attention (MultiHea  (None, 38, 1)       7169        ['layer_normalization[0][0]',    
 dAttention)                                                      'layer_normalization[0][0]']    
                                                                                              

 mbda)                                                            'tf.__operators__.add_5[0][0]'] 
                                                                                                  
 layer_normalization_7 (LayerNo  (None, 38, 1)       2           ['tf.__operators__.add_6[0][0]'] 
 rmalization)                                                                                     
                                                                                                  
 conv1d_6 (Conv1D)              (None, 38, 4)        8           ['layer_normalization_7[0][0]']  
                                                                                                  
 dropout_7 (Dropout)            (None, 38, 4)        0           ['conv1d_6[0][0]']               
                                                                                                  
 conv1d_7 (Conv1D)              (None, 38, 1)        5           ['dropout_7[0][0]']              
          

Epoch 23/200
50/50 [==============================] - 1s 21ms/step - loss: 0.1993 - mean_absolute_error: 0.4041 - auc: 0.7564 - binary_accuracy: 0.7003 - val_loss: 0.2164 - val_mean_absolute_error: 0.4357 - val_auc: 0.6932 - val_binary_accuracy: 0.6247
Epoch 24/200
50/50 [==============================] - 1s 20ms/step - loss: 0.1945 - mean_absolute_error: 0.3995 - auc: 0.7692 - binary_accuracy: 0.7084 - val_loss: 0.2158 - val_mean_absolute_error: 0.4325 - val_auc: 0.6897 - val_binary_accuracy: 0.6524
Epoch 25/200
50/50 [==============================] - 1s 20ms/step - loss: 0.1964 - mean_absolute_error: 0.3989 - auc: 0.7639 - binary_accuracy: 0.7009 - val_loss: 0.2170 - val_mean_absolute_error: 0.4350 - val_auc: 0.6912 - val_binary_accuracy: 0.6297
Epoch 26/200
50/50 [==============================] - 1s 20ms/step - loss: 0.1937 - mean_absolute_error: 0.3958 - auc: 0.7697 - binary_accuracy: 0.7021 - val_loss: 0.2165 - val_mean_absolute_error: 0.4327 - val_auc: 0.6921 - val_binary_accur

50/50 [==============================] - 1s 16ms/step - loss: 0.1512 - mean_absolute_error: 0.3126 - auc: 0.8611 - binary_accuracy: 0.7979 - val_loss: 0.2004 - val_mean_absolute_error: 0.3816 - val_auc: 0.7637 - val_binary_accuracy: 0.7053
Epoch 88/200
50/50 [==============================] - 1s 16ms/step - loss: 0.1524 - mean_absolute_error: 0.3146 - auc: 0.8597 - binary_accuracy: 0.7846 - val_loss: 0.1996 - val_mean_absolute_error: 0.3845 - val_auc: 0.7535 - val_binary_accuracy: 0.6977
Epoch 89/200
50/50 [==============================] - 1s 16ms/step - loss: 0.1476 - mean_absolute_error: 0.3101 - auc: 0.8690 - binary_accuracy: 0.8042 - val_loss: 0.1990 - val_mean_absolute_error: 0.3822 - val_auc: 0.7560 - val_binary_accuracy: 0.7053
Epoch 90/200
50/50 [==============================] - 1s 16ms/step - loss: 0.1472 - mean_absolute_error: 0.3076 - auc: 0.8681 - binary_accuracy: 0.8004 - val_loss: 0.1994 - val_mean_absolute_error: 0.3817 - val_auc: 0.7574 - val_binary_accuracy: 0.6927
E

In [7]:
#test results

In [8]:
test_x = data[split_point:]
test_y = targets[split_point:]

#n_classes = len(np.unique(train_y))

test_x = test_x.reshape((test_x.shape[0], test_x.shape[1], 1))
test_y = test_y.reshape(-1,1)

#remove win or lose reason labels (col 38 is first win reason)
test_x = test_x[:,:ace_col+1]
#for i in range(test_x.shape[0]):
#    test_x[i,56:] = 0
#print(test_x.shape)

#remove all rounds with aces
a = test_x.shape[0]
i=0
while(i<a):
    if(test_x[i,ace_col] == 0):
        i+=1
        continue
    else:
        test_x = np.delete(test_x, i, 0)
        test_y = np.delete(test_y, i, 0)
        a-=1
        if(i<=split_point):
            split_point-=1
        continue
test_x = test_x[:,:ace_col]

#remove all rounds with win or lose labels
#a = test_x.shape[0]
#i=0
#while(i<a):
#    if(np.array_equal(test_x[i,55:], np.zeros((16,1)))):
#        i+=1
#        continue
#    else:
#        test_x = np.delete(test_x, i, 0)
#        test_y = np.delete(test_y, i, 0)
#        a-=1
#        continue

#print(test_x.shape)
#print(test_y.shape)

In [9]:
evaluation = model.evaluate(test_x, test_y)

val_pred = model.predict(x=test_x)
val_pred = val_pred.reshape(val_pred.shape[0])
val_true = test_y
print("")
print("Validation Brier Score:")
print(brier_score_loss(y_true=val_true, y_prob=val_pred))

13/13 [==============================] - 0s 8ms/step - loss: 0.1809 - mean_absolute_error: 0.3557 - auc: 0.8046 - binary_accuracy: 0.7268

Validation Brier Score:
0.18089256350952884


In [10]:
pred = model.predict(test_x)

In [11]:
print(pred)
print(test_y)

[[0.6435586 ]
 [0.28178334]
 [0.479329  ]
 [0.2600587 ]
 [0.6329782 ]
 [0.5966262 ]
 [0.32378113]
 [0.61704946]
 [0.417957  ]
 [0.47212136]
 [0.00383495]
 [0.9893833 ]
 [0.26526558]
 [0.9893833 ]
 [0.4278333 ]
 [0.3837011 ]
 [0.8143035 ]
 [0.36076063]
 [0.9893833 ]
 [0.39141598]
 [0.85522133]
 [0.6022676 ]
 [0.3958684 ]
 [0.5109517 ]
 [0.60757816]
 [0.41435602]
 [0.84785414]
 [0.9052437 ]
 [0.38363448]
 [0.22125213]
 [0.06113416]
 [0.38896298]
 [0.746829  ]
 [0.38965085]
 [0.66818655]
 [0.46344396]
 [0.75499225]
 [0.46139196]
 [0.2892806 ]
 [0.4248227 ]
 [0.505048  ]
 [0.63972807]
 [0.21049611]
 [0.9154691 ]
 [0.29957646]
 [0.4781151 ]
 [0.21066858]
 [0.28403002]
 [0.8681585 ]
 [0.523142  ]
 [0.50750303]
 [0.4615775 ]
 [0.33515516]
 [0.9893833 ]
 [0.7367187 ]
 [0.4247359 ]
 [0.867149  ]
 [0.9878974 ]
 [0.25220343]
 [0.5960123 ]
 [0.63018346]
 [0.5570171 ]
 [0.680409  ]
 [0.76454705]
 [0.2558064 ]
 [0.790987  ]
 [0.8535999 ]
 [0.4699444 ]
 [0.53238   ]
 [0.54515517]
 [0.29635277]
 [0.95